# 📊 02. Feature Engineering Analysis

**Lab 02: Parsing & Reference Matching**  
**MSSV:** 23120067 - Lê Minh Nhật  
**Mục tiêu:** Phân tích và justify các features cho Reference Matching (Yêu cầu 2.2.3)

---

## 📋 Nội dung Notebook

1. [Setup & Import](#1-setup--import)
2. [Data Preparation](#2-data-preparation)
3. [Feature Groups Analysis](#3-feature-groups-analysis)
4. [Feature Correlation Analysis](#4-feature-correlation-analysis)
5. [Feature Importance Justification](#5-feature-importance-justification)
6. [Summary & Conclusions](#6-summary--conclusions)

---

## Yêu cầu từ text2.txt (Section 2.2.3)

> "Students must perform feature engineering to construct reasonable features for the model. 
> Students must **justify the creation of each feature** in the final report, explaining the underlying idea."

### Feature Groups (19+ features):
| Group | Features | Purpose |
|-------|----------|---------|
| Title | 5 | Primary matching signal |
| Author | 5 | Identity verification |
| Year | 4 | Temporal filtering |
| Text | 5 | Deep content matching |
| Hierarchy | 5+ | Citation context |